# LDA - XGBoost Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = Dataset(full_data_path=CLEANED_DATASET_PATH,
                  from_scratch=False,
                  split_sizes=[10000, 4232, 4232])
dataset.build()

X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

Data loaded from dataset/cleaned_dataset.pkl


In [4]:
# Re-split the dataset into training, validation, and test sets
# X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
# X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [5]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [6]:
# N_TOPICS = 10
# print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
# lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
# X_train_topics = lda.fit_transform(X_train_vector)
# X_val_topics = lda.transform(X_val_counts)
# X_test_topics = lda.transform(X_test_counts)
# print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

## Perform Non-negative Matrix Factorization on the training set

In [7]:
N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 20 topics
Done performing Non-negative Matrix Factorization for 20 topics


## Train the XGBoost model

In [8]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        # Suggest values for the hyperparameters
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
        max_depth = trial.suggest_int("max_depth", 3, 20)
        subsample = trial.suggest_float("subsample", 0.5, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)

        # Create an XGBoost classifier model with suggested parameters
        model = xgb.XGBClassifier(
            objective="binary:logistic",
            n_estimators=n_estimators,    # Number of trees
            learning_rate=learning_rate,  # Learning rate
            max_depth=max_depth,          # Depth of the trees
            subsample=subsample,          # Subsampling of the training instances
            colsample_bytree=colsample_bytree,  # Subsampling of columns for each tree
            seed=RANDOM_SEED,             # Seed for reproducibility
            use_label_encoder=False,      # Disable label encoder warning
            eval_metric="logloss")

        # Fit the model on the training data
        model.fit(X_train, Y_train)

        # Predict the labels on the test set
        Y_pred = model.predict(X_test)

        # Calculate the accuracy
        accuracy = accuracy_score(Y_test, Y_pred)
        return accuracy
    return objective

In [9]:
# Create a study object
# study = optuna.create_study(direction="maximize", study_name=f"LDA_XGBoost_Pipeline")
study = optuna.create_study(direction="maximize", study_name=f"NMF_XGBoost_Pipeline")


# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 23:30:47,894] A new study created in memory with name: LDA_XGBoost_Pipeline
[I 2024-04-30 23:30:57,445] Trial 3 finished with value: 0.6027882797731569 and parameters: {'n_estimators': 207, 'learning_rate': 0.0008820168556655033, 'max_depth': 7, 'subsample': 0.5921306411205443, 'colsample_bytree': 0.6469691352327644}. Best is trial 3 with value: 0.6027882797731569.
[I 2024-04-30 23:31:01,378] Trial 11 finished with value: 0.6784026465028355 and parameters: {'n_estimators': 257, 'learning_rate': 0.004176049323972921, 'max_depth': 8, 'subsample': 0.9829841975557365, 'colsample_bytree': 0.5606966638435017}. Best is trial 11 with value: 0.6784026465028355.
[I 2024-04-30 23:31:06,002] Trial 2 finished with value: 0.6838374291115312 and parameters: {'n_estimators': 271, 'learning_rate': 0.004818178657155203, 'max_depth': 10, 'subsample': 0.6218578186581494, 'colsample_bytree': 0.6300211479070421}. Best is trial 2 with value: 0.6838374291115312.
[I 2024-04-30 23:31:08,806] Trial

Best Accuracy: 0.6866729678638941
Best hyperparameters: {'n_estimators': 538, 'learning_rate': 0.005369262373720238, 'max_depth': 12, 'subsample': 0.7538188814682288, 'colsample_bytree': 0.5681588674515927}


In [10]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]
best_max_depth = study.best_trial.params["max_depth"]
best_subsample = study.best_trial.params["subsample"]
best_colsample_bytree = study.best_trial.params["colsample_bytree"]

model = xgb.XGBClassifier(n_estimators=best_n_estimators, 
                                 learning_rate=best_learning_rate,
                                 max_depth=best_max_depth,
                                 random_state=RANDOM_SEED)
model.fit(X_train_topics, Y_train)
Y_pred = model.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
# print(f"Best LDA-XGBoost Model Accuracy: {score}")
print(f"Best NMF-XGBoost Model Accuracy: {score}")

Best NMF-XGBoost Model Accuracy: 0.655718336483932
